# Part 01
### Optimize SVM Parameters (C and Gamma) on RBF Kernel

In [1]:
# imports
import random
import numpy
import math
import operator

from deap import base, creator, tools, algorithms, benchmarks

from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.datasets import load_boston
from sklearn import metrics

import matplotlib.pyplot as plt

In [2]:
db = load_boston()

In [3]:
creator.create("Fitness", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.Fitness, speed=list, smin=0.0, smax=1.0, best=1.0)

In [5]:
toolbox = base.Toolbox()

c_min = math.pow(2, -15)
c_max = math.pow(2, 3)
g_min = math.pow(2, -5)
g_max = math.pow(2, 15)

pmin = -100
pmax = 100

toolbox.register("c", random.uniform, c_min, c_max)
toolbox.register("gamma", random.uniform, g_min, g_max)
toolbox.register("individual", tools.initCycle, creator.Individual, (toolbox.c, toolbox.gamma), 1)
toolbox.register("population_ga", tools.initRepeat, list, toolbox.individual)

In [6]:
def evaluate(individual):
    # X_train, X_test, y_train, y_test = train_test_split(db.data, db.target)
    X = db.data
    y = db.target
    
    clf = SVR(kernel='rbf', C=individual[0], gamma=individual[1])
    
    scores = cross_val_score(clf, X, y, cv=5)
    acc = scores.mean()
    
    return acc,

In [7]:
def custom_mutation(ind):
    return ind,

In [9]:
toolbox.register("evaluate", evaluate_ga)
toolbox.register("mutate", custom_mutation)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("select", tools.selTournament, tournsize=3)

In [10]:
def apply_ga():
    ngen = 1000
    # mu = 200
    # lam = 20
    cxpb = 0.7
    mutpb = 0.2
    
    pop = toolbox.population_ga(n=10)
    hof = tools.ParetoFront()
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)
    
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=cxpb, mutpb=mutpb, ngen=ngen, 
                                       stats=stats, halloffame=hof, verbose=False)
    
    return pop, log, hof

In [11]:
if __name__ == "__main__":
    results_ga = apply_ga()
    print(results_ga[1])

gen 	nevals	avg      	std        	min      	max      
0   	10    	-0.589193	0.023934   	-0.629306	-0.569962
1   	7     	-0.570299	0.000678358	-0.5723  	-0.569962
2   	9     	-0.569986	7.01824e-05	-0.570196	-0.569962
3   	5     	-0.569962	1.11022e-16	-0.569962	-0.569962
4   	9     	-0.569962	1.11022e-16	-0.569962	-0.569962
5   	8     	-0.569962	1.11022e-16	-0.569962	-0.569962
6   	10    	-0.569962	1.11022e-16	-0.569962	-0.569962
7   	6     	-0.569962	1.11022e-16	-0.569962	-0.569962
8   	4     	-0.569962	1.11022e-16	-0.569962	-0.569962
9   	7     	-0.569962	1.11022e-16	-0.569962	-0.569962
10  	7     	-0.569962	1.11022e-16	-0.569962	-0.569962
11  	7     	-0.569962	1.11022e-16	-0.569962	-0.569962
12  	6     	-0.569962	1.11022e-16	-0.569962	-0.569962
13  	4     	-0.569962	1.11022e-16	-0.569962	-0.569962
14  	8     	-0.569962	1.11022e-16	-0.569962	-0.569962
15  	6     	-0.569962	1.11022e-16	-0.569962	-0.569962
16  	8     	-0.569962	1.11022e-16	-0.569962	-0.569962
17  	10    	-0.569962	1.1102